In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import math
import time

########################
# Configuration
########################
ticker = 'AMGN'
start_date = '2020-01-01'
end_date = '2024-01-01'
1
# Hyperparameters
n_steps = 2
window_size = 5
step_size = 2
initial_epochs = 100
update_epochs = 100
initial_lr = 0.0005
update_lr = 0.00005
train_ratio = 0.9
n_units = 16
batch_size = 4

# If you change features or other logic, do so below
if not os.path.exists("rolling_error_analysis_results"):
    os.makedirs("rolling_error_analysis_results")


def load_full_data_helper(ticker, start_date, end_date, window_size):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    if stock_data.empty:
        return None, None, None

    # Next day prediction setup
    stock_data['Close_next_day'] = stock_data['Close'].shift(-1)
    stock_data.dropna(subset=['Close_next_day'], inplace=True)

    # Simple features
    X_feat = stock_data[['Open', 'High', 'Low']]
    y_raw = stock_data['Close_next_day'].values
    dates = stock_data['Date'].values

    if len(dates) < window_size:
        return None, None, None
    return X_feat.values, y_raw, dates


def load_full_data(ticker, start_date, end_date, window_size):
    X_feat, y_raw, dates = load_full_data_helper(ticker, start_date, end_date, window_size)
    if X_feat is None:
        return None, None, None
    return X_feat, y_raw, dates


def extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps):
    end_idx = start_idx + window_size
    if end_idx > len(X_full):
        return None, None, None, None, None, None

    X_window = X_full[start_idx:end_idx]
    y_window = y_full[start_idx:end_idx]
    dates_window = dates_full[start_idx:end_idx]

    def lstm_split(dataX, dataY, n_steps):
        X, y = [], []
        for i in range(len(dataX)-n_steps+1):
            X.append(dataX[i:i+n_steps])
            y.append(dataY[i+n_steps-1])
        return np.array(X), np.array(y)

    X_samples, y_samples = lstm_split(X_window, y_window, n_steps)
    sample_dates = dates_window[n_steps-1:]

    feat_scaler = RobustScaler()
    target_scaler = RobustScaler()
    X_flat = X_window.reshape(len(X_window), -1)
    feat_scaler.fit(X_flat)
    y_window_reshaped = y_window.reshape(-1,1)
    target_scaler.fit(y_window_reshaped)

    X_scaled = feat_scaler.transform(X_flat).reshape(X_window.shape)
    y_scaled = target_scaler.transform(y_window_reshaped).flatten()
    X_samples_scaled, y_samples_scaled = lstm_split(X_scaled, y_scaled, n_steps)

    return X_samples_scaled, y_samples_scaled, sample_dates, feat_scaler, target_scaler, (X_window, y_window, dates_window)


def create_lstm_model(n_units, learning_rate, n_steps, n_features, n_layers=1):
    inputs = Input(shape=(n_steps, n_features))
    x = LSTM(n_units, activation='relu', return_sequences=(n_layers > 1))(inputs)
    if n_layers > 1:
        x = LSTM(n_units, activation='relu')(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    # We'll compile after loading weights if needed
    return model


def train_on_window(X_samples, y_samples, feat_scaler, target_scaler,
                    initial_training=False, prev_weights=None):
    total_samples = len(X_samples)
    global train_ratio
    train_count = int(math.floor(total_samples * train_ratio))
    if train_count >= total_samples:
        train_count = total_samples - 1

    X_train, y_train = X_samples[:train_count], y_samples[:train_count]
    X_val, y_val = X_samples[train_count:], y_samples[train_count:]

    if X_train.size == 0 or X_val.size == 0:
        return None, np.nan

    n_steps_local = X_train.shape[1]
    n_features = X_train.shape[2]

    if initial_training:
        epochs = initial_epochs
        lr = initial_lr
    else:
        epochs = update_epochs
        lr = update_lr

    model = create_lstm_model(n_units, lr, n_steps_local, n_features, n_layers=1)

    if prev_weights is not None:
        model.load_weights(prev_weights)

    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))

    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, shuffle=False,
              validation_data=(X_val, y_val), callbacks=[es])

    y_val_pred_scaled = model.predict(X_val, verbose=0)
    y_val_unscaled = target_scaler.inverse_transform(y_val.reshape(-1,1)).flatten()
    y_val_pred_unscaled = target_scaler.inverse_transform(y_val_pred_scaled).flatten()
    val_mape = mean_absolute_percentage_error(y_val_unscaled, y_val_pred_unscaled) * 100

    return model, val_mape


def rolling_training_early_stopping(ticker, window_size, step_size, n_steps):
    X_full, y_full, dates_full = load_full_data(ticker, start_date, end_date, window_size)
    if X_full is None:
        print("[WARN] Not enough data.")
        return

    start_time = time.time()

    start_idx = 0
    window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
    if window_data[0] is None:
        print("[WARN] Initial window not valid.")
        return
    X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

    print("[INFO] Training initial window with Early Stopping...")
    model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=True, prev_weights=None)
    print(f"[RESULT] Initial window validation MAPE: {val_mape:.2f}%")

    weights_path = "rolling_error_analysis_results/cost_initial_es.weights.h5"
    model.save_weights(weights_path)

    predictions = []
    actuals = []
    pred_dates = []

    iteration = 1
    while True:
        start_idx += step_size
        window_data = extract_window(X_full, y_full, dates_full, start_idx, window_size, n_steps)
        if window_data[0] is None:
            print("[INFO] No more windows can be extracted.")
            break
        X_samples, y_samples, sample_dates, feat_scaler, target_scaler, (Xw, yw, dw) = window_data

        print(f"\n[INFO] Iteration {iteration}: training on new window with Early Stopping...")
        model, val_mape = train_on_window(X_samples, y_samples, feat_scaler, target_scaler, initial_training=False, prev_weights=weights_path)
        print(f"[RESULT] Window validation MAPE: {val_mape:.2f}%")

        weights_path = f"rolling_error_analysis_results/cost_iteration_{iteration}_es.weights.h5"
        model.save_weights(weights_path)

        X_test_last = X_samples[-1:]
        y_test_last = y_samples[-1:]
        y_pred_scaled = model.predict(X_test_last, verbose=0)
        y_test_unscaled = target_scaler.inverse_transform(y_test_last.reshape(-1,1)).flatten()
        y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled).flatten()

        predictions.append(y_pred_unscaled[0])
        actuals.append(y_test_unscaled[0])
        pred_dates.append(sample_dates[-1])

        test_mape = mean_absolute_percentage_error(y_test_unscaled, y_pred_unscaled)*100
        print(f"[INFO] Predicted last sample of window: Actual={y_test_unscaled[0]:.2f}, Pred={y_pred_unscaled[0]:.2f}, MAPE={test_mape:.2f}%")

        iteration += 1

    df_results = pd.DataFrame({
        'Date': pred_dates,
        'Actual': actuals,
        'Predicted': predictions
    })
    df_results['Absolute_Error'] = np.abs(df_results['Actual'] - df_results['Predicted'])
    df_results['APE'] = df_results['Absolute_Error'] / np.abs(df_results['Actual'])
    df_results['MAPE'] = df_results['APE'] * 100.0

    df_results.to_csv("rolling_error_analysis_results/cost_rolling_results_es.csv", index=False)
    overall_mape = df_results['MAPE'].mean()
    print(f"\n[INFO] Rolling training with Early Stopping completed for {ticker}. Overall MAPE: {overall_mape:.2f}%")
    elapsed = time.time() - start_time
    print(f"[INFO] Total elapsed time: {elapsed:.2f}s")

    worst_10 = df_results.sort_values('MAPE', ascending=False).head(10)
    print("[INFO] Top 10 worst predictions in rolling scenario (Early Stopping):")
    print(worst_10[['Date', 'Actual', 'Predicted', 'MAPE']])

if __name__ == "__main__":
    # Just run with the current top-level defined parameters:
    rolling_training_early_stopping(ticker, window_size, step_size, n_steps)


[*********************100%%**********************]  1 of 1 completed
[INFO] Training initial window with Early Stopping...
[RESULT] Initial window validation MAPE: 0.14%

[INFO] Iteration 1: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.56%
[INFO] Predicted last sample of window: Actual=236.92, Pred=238.25, MAPE=0.56%

[INFO] Iteration 2: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.22%
[INFO] Predicted last sample of window: Actual=241.70, Pred=238.75, MAPE=1.22%

[INFO] Iteration 3: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.20%
[INFO] Predicted last sample of window: Actual=241.49, Pred=241.00, MAPE=0.20%

[INFO] Iteration 4: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.72%
[INFO] Predicted last sample of window: Actual=236.75, Pred=240.83, MAPE=1.72%

[INFO] Iteration 5: training on new window with Early Stopping...
[RESULT] Window validat

[RESULT] Window validation MAPE: 4.47%
[INFO] Predicted last sample of window: Actual=242.43, Pred=231.60, MAPE=4.47%

[INFO] Iteration 38: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.28%
[INFO] Predicted last sample of window: Actual=235.69, Pred=236.35, MAPE=0.28%

[INFO] Iteration 39: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.89%
[INFO] Predicted last sample of window: Actual=230.98, Pred=237.66, MAPE=2.89%

[INFO] Iteration 40: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.15%
[INFO] Predicted last sample of window: Actual=236.36, Pred=236.00, MAPE=0.15%

[INFO] Iteration 41: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.19%
[INFO] Predicted last sample of window: Actual=233.33, Pred=233.78, MAPE=0.19%

[INFO] Iteration 42: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.59%
[INFO] Predicted last sample of

[RESULT] Window validation MAPE: 1.93%
[INFO] Predicted last sample of window: Actual=258.12, Pred=253.14, MAPE=1.93%

[INFO] Iteration 83: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.94%
[INFO] Predicted last sample of window: Actual=248.40, Pred=250.74, MAPE=0.94%

[INFO] Iteration 84: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.76%
[INFO] Predicted last sample of window: Actual=245.57, Pred=247.44, MAPE=0.76%

[INFO] Iteration 85: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.06%
[INFO] Predicted last sample of window: Actual=243.21, Pred=243.06, MAPE=0.06%

[INFO] Iteration 86: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.98%
[INFO] Predicted last sample of window: Actual=248.35, Pred=245.92, MAPE=0.98%

[INFO] Iteration 87: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.14%
[INFO] Predicted last sample of

[RESULT] Window validation MAPE: 1.63%
[INFO] Predicted last sample of window: Actual=238.49, Pred=234.60, MAPE=1.63%

[INFO] Iteration 127: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.09%
[INFO] Predicted last sample of window: Actual=236.59, Pred=236.80, MAPE=0.09%

[INFO] Iteration 128: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.65%
[INFO] Predicted last sample of window: Actual=241.66, Pred=237.67, MAPE=1.65%

[INFO] Iteration 129: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.12%
[INFO] Predicted last sample of window: Actual=248.27, Pred=245.49, MAPE=1.12%

[INFO] Iteration 130: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.12%
[INFO] Predicted last sample of window: Actual=252.03, Pred=249.21, MAPE=1.12%

[INFO] Iteration 131: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.77%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 0.13%
[INFO] Predicted last sample of window: Actual=252.38, Pred=252.05, MAPE=0.13%

[INFO] Iteration 171: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.10%
[INFO] Predicted last sample of window: Actual=248.63, Pred=251.38, MAPE=1.10%

[INFO] Iteration 172: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.05%
[INFO] Predicted last sample of window: Actual=251.01, Pred=250.89, MAPE=0.05%

[INFO] Iteration 173: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.79%
[INFO] Predicted last sample of window: Actual=242.00, Pred=248.75, MAPE=2.79%

[INFO] Iteration 174: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.41%
[INFO] Predicted last sample of window: Actual=235.31, Pred=240.98, MAPE=2.41%

[INFO] Iteration 175: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.77%
[INFO] Predicted last samp

...

[RESULT] Window validation MAPE: 2.52%
[INFO] Predicted last sample of window: Actual=248.28, Pred=254.53, MAPE=2.52%

[INFO] Iteration 303: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.08%
[INFO] Predicted last sample of window: Actual=245.44, Pred=248.09, MAPE=1.08%

[INFO] Iteration 304: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.40%
[INFO] Predicted last sample of window: Actual=245.48, Pred=246.47, MAPE=0.40%

[INFO] Iteration 305: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.65%
[INFO] Predicted last sample of window: Actual=240.01, Pred=243.97, MAPE=1.65%

[INFO] Iteration 306: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.65%
[INFO] Predicted last sample of window: Actual=235.70, Pred=239.60, MAPE=1.65%

[INFO] Iteration 307: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.13%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 0.84%
[INFO] Predicted last sample of window: Actual=229.03, Pred=230.96, MAPE=0.84%

[INFO] Iteration 347: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 5.37%
[INFO] Predicted last sample of window: Actual=245.44, Pred=232.27, MAPE=5.37%

[INFO] Iteration 348: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.36%
[INFO] Predicted last sample of window: Actual=251.66, Pred=245.72, MAPE=2.36%

[INFO] Iteration 349: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.60%
[INFO] Predicted last sample of window: Actual=252.93, Pred=251.40, MAPE=0.60%

[INFO] Iteration 350: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.44%
[INFO] Predicted last sample of window: Actual=248.19, Pred=251.76, MAPE=1.44%

[INFO] Iteration 351: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.02%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 0.06%
[INFO] Predicted last sample of window: Actual=241.03, Pred=241.18, MAPE=0.06%

[INFO] Iteration 391: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.77%
[INFO] Predicted last sample of window: Actual=234.22, Pred=240.70, MAPE=2.77%

[INFO] Iteration 392: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.63%
[INFO] Predicted last sample of window: Actual=238.24, Pred=239.74, MAPE=0.63%

[INFO] Iteration 393: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.05%
[INFO] Predicted last sample of window: Actual=237.62, Pred=237.73, MAPE=0.05%

[INFO] Iteration 394: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.58%
[INFO] Predicted last sample of window: Actual=234.45, Pred=235.81, MAPE=0.58%

[INFO] Iteration 395: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.90%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 0.51%
[INFO] Predicted last sample of window: Actual=229.66, Pred=228.50, MAPE=0.51%

[INFO] Iteration 435: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.96%
[INFO] Predicted last sample of window: Actual=224.73, Pred=226.89, MAPE=0.96%

[INFO] Iteration 436: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.10%
[INFO] Predicted last sample of window: Actual=221.31, Pred=223.74, MAPE=1.10%

[INFO] Iteration 437: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.09%
[INFO] Predicted last sample of window: Actual=222.02, Pred=221.81, MAPE=0.09%

[INFO] Iteration 438: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.62%
[INFO] Predicted last sample of window: Actual=225.88, Pred=222.23, MAPE=1.62%

[INFO] Iteration 439: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 1.77%
[INFO] Predicted last samp

[RESULT] Window validation MAPE: 0.99%
[INFO] Predicted last sample of window: Actual=269.71, Pred=272.37, MAPE=0.99%

[INFO] Iteration 479: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.59%
[INFO] Predicted last sample of window: Actual=263.19, Pred=264.74, MAPE=0.59%

[INFO] Iteration 480: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.00%
[INFO] Predicted last sample of window: Actual=260.84, Pred=260.84, MAPE=0.00%

[INFO] Iteration 481: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.59%
[INFO] Predicted last sample of window: Actual=269.86, Pred=262.87, MAPE=2.59%

[INFO] Iteration 482: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 0.14%
[INFO] Predicted last sample of window: Actual=271.11, Pred=270.74, MAPE=0.14%

[INFO] Iteration 483: training on new window with Early Stopping...
[RESULT] Window validation MAPE: 2.68%
[INFO] Predicted last samp